# 데이터셋 구축

In [1]:
import requests
import pandas as pd
from pykrx import stock
from datetime import datetime
from bs4 import BeautifulSoup as bs

## 1. 코스피 50 종목 가져오기

In [2]:
## 오늘 날짜 가져오기 ##
today = datetime.today().isoformat()[:10]
today = today.replace('-', '')
today

'20230623'

In [3]:
## 코스피 50 종목 가져오기 ##

# 지수 티커 확인: https://github.com/sharebook-kr/pykrx#212-%EC%9D%B8%EB%8D%B1%EC%8A%A4-%EC%A1%B0%ED%9A%8C-api
tickers = stock.get_index_portfolio_deposit_file("1035")

stocks = []
for ticker in tickers:
    name = stock.get_market_ticker_name(ticker)
    stocks.append(name)

print(f'종목 개수: {len(stocks)}')
print(f'10개 종목만 출력해보기: {stocks[:10]}')

종목 개수: 50
10개 종목만 출력해보기: ['삼성전자', 'LG에너지솔루션', 'SK하이닉스', '삼성바이오로직스', 'LG화학', '삼성SDI', '현대차', '기아', 'POSCO홀딩스', 'NAVER']


## 2. 종목별 뉴스 크롤링

In [4]:
# 뉴스 크롤링 시작 날짜
start_date = '2023.06.01'

# 뉴스 크롤링 종료 날짜
end_date = '2023.06.15'

headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3)'}

In [5]:
stock_news = pd.DataFrame()

for stock in stocks:

    # 시작 페이지
    page = 1 

    while True:
        url = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query={}&sort=0&photo=3&field=0&pd=1&ds={}&de={}&cluster_rank=27&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:1w,a:all&start={}'.format(stock, start_date, end_date, page)
        html = requests.get(url, headers = headers).text
        soup = bs(html, 'html.parser')

        news_list = soup.select_one('#main_pack > section > div > div.group_news > ul')
        try:
            news_list = news_list.find_all('li', 'bx')
        except:
            break
            
        if len(news_list) == 10:
            pass
        else:
            break
        
        for n in news_list:
            title = n.find('a', "news_tit").text
            press = n.find('a', "info press").text
            all_element = pd.DataFrame({'stock':[stock],'press':[press],'title':[title]})
            stock_news = pd.concat([stock_news, all_element])

        # 다음 페이지
        page += 10 

## 3. 결과 저장

In [6]:
stock_news.reset_index(drop = True, inplace = True)
stock_news.to_excel('data/stock_news_result_{}.xlsx'.format(start_date + '-' + end_date))

In [7]:
stock_news

,stock,press,title
0,삼성전자,파이낸셜뉴스,"나흘간 1조3000억 팔아치운 외국인 ""삼성전자는 더 담자"""
1,삼성전자,한국경제,삼성전자·자이·대한항공·하이카·신한銀 '19년째 사랑받는 1위'
2,삼성전자,전자신문언론사 선정,"삼성전자, 美 텍사스에 네트워크혁신센터 구축"
3,삼성전자,아시아경제,"송봉섭 삼성전자서비스 대표, 한국능률협회컨설팅 '특별 공헌상' 수상"
4,삼성전자,대전일보,"KAIST-삼성전자, 반도체 인재 양성 지원 확대"
...,...,...,...
3755,아모레퍼시픽,세계일보,"제주 ‘미래 먹거리’ 바이오, 연구·생산·유통 밸류체인 구축한다"
3756,아모레퍼시픽,머니투데이,[MT시평]중국투자백서를 만들자
3757,아모레퍼시픽,한국경제,신동원의 첫번째 '깡' 스낵…선친 유산 잇는다
3758,아모레퍼시픽,동아일보,"제주도, 바이오산업 경쟁력 확보 위해 육성전략 마련"
